In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Change Directory
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/R_Python_Stock/')
os.getcwd()

'/content/drive/My Drive/Colab Notebooks/R_Python_Stock'

In [3]:
import pandas as pd

In [4]:
def parse_StockAsia(page):
  from bs4 import BeautifulSoup
  import requests
  soup = BeautifulSoup(page.text, 'html.parser')
  if page.status_code==200:

      #Getcursor
      cursor = soup.find('div',  {"class":"xpanel-group xgrey-bg xbottom5"}).find('input',  {"name":"cursor"})
      cursor_value = cursor['value']
      cursor_value = cursor_value.replace("==","%3D%3D")

      #Parse HTML
      container = soup.find('div', {"class":"container-fluid"})
      rows = container.findAll('div', {"class":"row","itemprop":"hasPart" })
      listNews=[]
      for row in rows:
        panel = row.find('div', {'class':'xpanel-group col-sm-6 xbottom10'}).find('div', {'class':'xpanel xpanel-default'})    
        panel_heading = panel.find('div', {'class':'xpanel-heading'})
        panel_heading_h2 = panel_heading.find('h2').find('a')
        heading = panel_heading_h2.text.strip()
        heading_href = "http://stocks.asia/"+panel_heading_h2['href']
        #print(heading +" "+ heading_href)
        publisher = panel_heading.find('div').find('span',{'itemprop':'publisher'}).find('span',{'class':'xsiteurl small'}).find('span',{'itemprop':'name'}).text
        language = panel_heading.find('div').find('span',{'itemprop':'inLanguage'}).text
        mentions = panel_heading.find('div').findAll('span',{'itemprop':'mentions'})
       
        
        tickerSymbolsAll=""
        for mention in mentions:
          tickerSymbols = mention.find('a').find('span',{'itemprop':'tickerSymbol'}).text
          tickerSymbolsAll = tickerSymbolsAll+"|"+tickerSymbols if tickerSymbolsAll!="" else tickerSymbols
          tickerSymbols_href = "http://stocks.asia"+mention.find('a')['href']
          #print(tickerSymbols+" "+tickerSymbols_href)
      # print(tickerSymbolsAll)

        panel_body = panel.find('div',{'class':'xpanel-body xno-vmargins'})
        panel_body_genres = panel_body.find('div',{'class':'text-muted small xvspace5'}).findAll('a')
        
        genreAll=""
        for panel_body_genre in panel_body_genres:  
          genre = panel_body_genre.find('span').text
          genreAll = genreAll+"|"+genre if genreAll!="" else genre
          genre_href = "http://stocks.asia"+panel_body_genre['href']
          #print(genre+" "+genre_href)
        #print(genreAll)

        panel_body_description = panel_body.find('div',{'itemprop':'description'}).text.split('\t')[1]
        #print(panel_body_description)
        panel_body_tanggal = panel_body.find('div',{'itemprop':'description'}).find('span').text[0:17]
        #print(panel_body_tanggal)

        listNews.append([heading,heading_href,publisher,language,tickerSymbolsAll,genreAll,panel_body_tanggal,panel_body_description])
  return [cursor_value,listNews]

In [5]:
from bs4 import BeautifulSoup
import requests
 
page = requests.get('https://stocks.asia/')
cursor_value,listNews =parse_StockAsia(page)
df_listNewsAllPage = pd.DataFrame(listNews)


In [ ]:

halamans=[10,20,30,40,50]#,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200]
#df_listNewsAllPage=pd.DataFrame()
for halaman in halamans:
  url = 'https://stocks.asia/?cursor='+cursor_value+'&cur_pos='+str(halaman)+'&next_pos='+str(int(halaman)+10)+'&front_page=1&op=next' 
  #print(url)
  page = requests.get(url)
  cursor_value,listNews =parse_StockAsia(page)
  df_listNews = pd.DataFrame(listNews)
  df_listNewsAllPage = pd.concat([df_listNewsAllPage,df_listNews],axis='rows')



In [7]:
df_listNewsAllPage.columns=['Heading','Heading_Href','Publisher','Language','Ticker','Topik','Date','Body']

In [64]:
df_listNewsAllPage = df_listNewsAllPage.reset_index(drop=True)
df_listNewsAllPage.head(3)

,Heading,Heading_Href,Publisher,Language,Ticker,Topik,Date,Body
0,Update Harga Komoditas dan Indeks | 15 Oktober...,http://stocks.asia/go?h=8a54cdb81cc458c41b0159...,Saham Online Indonesia,en,TLKM,Kondisi Pasar|Blog,15 Oct 2020 07:20,Stocks close lower for a second day amid coron...
1,Ini sederet sentimen IHSG dalam jangka pendek,http://stocks.asia/go?h=e0e7a0c56c3a29ac11239c...,Kontan,id,IHSG,Analisa Fundamental|Foreign Flow|Makroekonomi|...,15 Oct 2020 07:15,Reporter: Ika Puspitasari Editor: Yudho Winart...
2,[PRE-ORDER] File Rekap Laporan Sembilan Bulan ...,http://stocks.asia/go?h=dd2f26f22be9f8fcea9236...,Saham Syariah Indonesia,id,BMRI,Analisa Fundamental|Berita IPO|Blog,15 Oct 2020 07:11,"Syariahsaham.com, CIANJUR-- Mulai pertengahan ..."


In [65]:
def calculate_sentiment(x):
  text = TextBlob(x[1])
  if x[0]=='id':
    text_translate = text.translate(from_lang='id', to='en')
  else:
    text_translate = text
  
  return [text_translate.sentiment.polarity,text_translate.sentiment.subjectivity]


In [66]:
sentiment_result = df_listNewsAllPage[['Language','Body']].apply(calculate_sentiment, axis='columns')

In [90]:
df_listNewsAllPage['polarity']=sentiment_result.apply(lambda x : x[0])
df_listNewsAllPage['subjectivity']=sentiment_result.apply(lambda x : x[1])

In [91]:
df_listNewsAllPage

,Heading,Heading_Href,Publisher,Language,Ticker,Topik,Date,Body,polarity,subjectivity
0,Update Harga Komoditas dan Indeks | 15 Oktober...,http://stocks.asia/go?h=8a54cdb81cc458c41b0159...,Saham Online Indonesia,en,TLKM,Kondisi Pasar|Blog,15 Oct 2020 07:20,Stocks close lower for a second day amid coron...,0.166667,0.166667
1,Ini sederet sentimen IHSG dalam jangka pendek,http://stocks.asia/go?h=e0e7a0c56c3a29ac11239c...,Kontan,id,IHSG,Analisa Fundamental|Foreign Flow|Makroekonomi|...,15 Oct 2020 07:15,Reporter: Ika Puspitasari Editor: Yudho Winart...,-0.047840,0.114506
2,[PRE-ORDER] File Rekap Laporan Sembilan Bulan ...,http://stocks.asia/go?h=dd2f26f22be9f8fcea9236...,Saham Syariah Indonesia,id,BMRI,Analisa Fundamental|Berita IPO|Blog,15 Oct 2020 07:11,"Syariahsaham.com, CIANJUR-- Mulai pertengahan ...",0.086429,0.302143
3,"Utang Menumpuk, Masyarakat Makin Melarat",http://stocks.asia/go?h=8665e9076b79416206cdcb...,CNN Indonesia,id,,Analisa Fundamental|Makroekonomi|Situs Berita,15 Oct 2020 07:10,"Jakarta, CNN Indonesia -- Bank Dunia (World Ba...",0.069444,0.173148
4,Corona Durjana! Prancis Sampai Tetapkan Darura...,http://stocks.asia/go?h=973659877c1419fefb843b...,CNBC Indonesia,id,,Makroekonomi|Situs Berita,15 Oct 2020 07:05,"Sentimen keempat, kali ini dari dalam negeri, ...",0.004167,0.180208
5,Corona Durjana! Prancis Sampai Tetapkan Darura...,http://stocks.asia/go?h=624c4b433220889ac4e135...,CNBC Indonesia,id,,Makroekonomi|Situs Berita,15 Oct 2020 07:05,Berikut sejumlah agenda dan rilis data yang te...,-0.050000,0.050000
6,"IHSG sudah reli delapan hari, waspadai aksi am...",http://stocks.asia/go?h=d4d6e3e5bcbf82aecae2ab...,Kontan,id,IHSG|ANTM|MEDC,Analisa Teknikal|Foreign Flow|Kondisi Pasar|Si...,15 Oct 2020 07:05,Reporter: Akhmad Suryahadi Editor: Herlina Kar...,-0.050000,0.200000
7,Corona Durjana! Prancis Sampai Tetapkan Darura...,http://stocks.asia/go?h=441bd97f50361a5b7144ad...,CNBC Indonesia,id,,Analisa Fundamental|Kondisi Pasar|Situs Berita,15 Oct 2020 07:05,"Beralih ke Wall Street, tiga indeks utama ditu...",-0.133460,0.600505
8,"5 Saham Anak BUMN Ngamuk Kemarin, Duh Nyesel G...",http://stocks.asia/go?h=d6bade807d96a06b0e293a...,CNBC Indonesia,id,TINS|BRIS|ANTM|AGRO|BBRI|IHSG|BBNI|UNTR|INCO|PTBA,Foreign Flow|Kondisi Pasar|Situs Berita,15 Oct 2020 07:05,"Jakarta, CNBC Indonesia- Indeks Harga Saham Ga...",-0.033333,0.166667
9,IHSG Diramal Loyo Jelang Rilis Neraca Perdagangan,http://stocks.asia/go?h=501e2cdcdb4d96127606bb...,CNN Indonesia,id,IHSG|ASII|BBCA|BBNI|SCMA|KAEF|TLKM|TBIG|AKRA,Analisa Teknikal|Kondisi Pasar|Situs Berita,15 Oct 2020 06:41,"Jakarta, CNN Indonesia -- Indeks Harga Saham G...",0.090000,0.303333


In [92]:
df_listNewsAllPage['polarity'].mean()

0.06386285031078359

In [93]:
df_listNewsAllPage['subjectivity'].mean()

0.35154489461500354